In [42]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys
import pickle
#PARAMS
p = 127
Fp = GF(p)
n = 128
k = 76
w = 9
r = n-k
t = 16
z = 7
Fz = GF(z)
lamb = 128
depth = log(t,2)

Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])


def generate_random_seed():
    set_random_seed()
    return initial_seed()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

In [43]:
#SEED TREE

class SeedTree:
    def __init__(self, depth, salt):
        if(depth>0):
            self.seed_m = None
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    def make_tree(self, seed_m):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            self.seed_m = seed_m
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree([self.seed_m], 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def compute_level(self, parents, level):
        return self.__create_new_level(parents, level) 

    def get_seed_path(self):
        if self.full and self.seed_path is not None:
            return self.seed_path

    def compute_seed_path(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            self.seed_path={}
            self.__compute_seed_path(self.depth, list_nodes)
        else:
            raise Exception('Albero non creato o foglia non valida')
        
    def get_required_leaves_from_seed_path(self, seed_path, leaves_indexes):
        leaves = self.__get_required_leaves_from_seed_path(seed_path)
        if list(leaves.keys())==leaves_indexes:
            return leaves
        else:
            raise Exception('Seed Path non corretto')
    

    def __make_tree(self, list_nodes, lev):
        if(lev<=self.depth):
            self.tree.append(list_nodes)
            list_children = self.__create_new_level(list_nodes, lev)
            self.__make_tree(list_children, lev+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents[0], '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
    
    def __recompute_new_level(self, parents, level):
        children = {}
        base = 2**level
        for i in parents.keys():
            sx, dx = self.__compute_children_value(parents[i], base+i)
            children[i*2]=sx
            children[(i*2)+1]=dx
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    def __compute_seed_path(self, depth, list_index):
        if(depth > 1):
            list_index_next_level= list(filter(lambda x: (x % 2 == 0 and x + 1 in list_index) or (x % 2 != 0 and x - 1 in list_index), list_index)) 
            list_index = list(filter(lambda x: x not in list_index_next_level, list_index)) 
            list_parent = sorted(list(set(map(lambda elem: (elem // 2), list_index_next_level)))) 
            if len(list_index)>0:
                self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_index)) 
            if len(list_parent)>0:
                self.__compute_seed_path(depth-1, list_parent) 
    
    def __get_required_leaves_from_seed_path(self, seed_path):
        if len(seed_path.keys())>1:
            index = min(seed_path.keys())
            current_nodes=seed_path.pop(index)
            recomputed_children = self.__recompute_new_level(current_nodes, index)
            children = seed_path[index+1]
            merged_children = {**children, **recomputed_children} 
            seed_path[index+1] = dict(sorted(merged_children.items()))
            return self.__get_required_leaves_from_seed_path(seed_path)
        else:
            index, current_nodes = seed_path.popitem()
            return current_nodes


            

In [44]:
seed_m = generate_random_seed() 
seed_salt = generate_random_seed()
salt = hash_and_get_seed(seed_salt)
seed_tree = SeedTree(depth, salt)
seed_tree.make_tree(seed_m)

index_1=[1,3,4,6,11,12,13,14]
seed_tree.compute_seed_path(index_1)
seed_path=seed_tree.get_seed_path()
#print(len(seed_path.keys()))
#for x in seed_path:
#    print(x, seed_path[x])
seed_path=seed_tree.get_seed_path()
leaves=seed_tree.get_required_leaves_from_seed_path(seed_path, index_1)
print(leaves)
for i in list(leaves.keys()):
    print(seed_tree.tree[-1][i] == leaves[i])



{1: 14492683306407510339492403244402670979, 3: 259043112404415006729641331794734026818, 4: 201024195140981406192890748623491276308, 6: 24799655684559250370482975741945786880, 11: 227403558405413200575832759269483585924, 12: 331391218317103122270789657720660627944, 13: 107742128141548397682276389693216430495, 14: 283556372636355988913394861768488018246}
True
True
True
True
True
True
True
True
